<a href="https://colab.research.google.com/github/MarciaFG/mobility_analytics/blob/main/CosineSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Collaboratory Initialization**

Author:Marcia Ferreira

Date: November 26, 2020

Input: 

Model: 

Output: 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Thu Nov 26 08:02:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.8 GB  | Proc size: 110.6 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


# Install required Drivers

In [ ]:
from tqdm import tqdm
%matplotlib inline
import pandas as pd, numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt
import io

# Loading the data


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv(io.StringIO(uploaded['cos_df_raw.txt'].decode('utf-8')))
df.head(10)

In [ ]:
summary = df.groupby(['grid_id', 'cl']).size().unstack().fillna(0)
summary.head(10)

pairwise = pd.DataFrame(
    squareform(pdist(summary, metric='cosine')),
    columns = summary.index,
    index = summary.index
)

# move to long form
long_form = pairwise.unstack()
# rename columns and turn into a dataframe
long_form.index.rename(['source', 'target'], inplace=True)

long_form = long_form.to_frame('cosine_distance').reset_index()

long_form['cosine_sim'] = 1- long_form['cosine_distance'] 
long_form['weight'] = long_form['cosine_sim'] 

# dropping ALL duplicte values 
result = long_form.sort_values('source', ascending=False).drop_duplicates('target').sort_index()
df_result = result.query("source != target")

# saving the results
df_result.to_csv('C:/Users/Lenovo/Desktop/orgfields/t1_edgelist_norm.txt', index=False)
